In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
%matplotlib inline
import networkx as nx 
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt 
%matplotlib inline 

N=8;

S0=3.34;

K=3.18 

sigma=0.68;
dt=3.5/365;

r = 0.02*dt;

a=(1+r)*(1-sigma*np.sqrt(dt))-1;
b=(1+r)*(1+sigma*np.sqrt(dt))-1;

p = (r-a)/(b-a)
q = (b-r)/(b-a)

X = np.empty(N+1, dtype=int)
for i in range(0,N+1):
    X[i]=i

def plot_tree(axarr,g):
    plt.figure(figsize=(20,10))
    Z = np.empty(3)
    global N
    for k in reversed(range(0,N)):
        for l in range(-k+1,k+3,2):
            Z[0]=g.node[(k,l)]['value']
            Z[1]=g.node[(k+1,l+1)]['value']
            Z[2]=3
            axarr.plot(X[k:k+2],Z[0:2],color='blue',marker='.',markersize = 10)
            Z[0]=g.node[(k,l)]['value']
            Z[1]=g.node[(k+1,l-1)]['value']
            Z[2]=3
            axarr.plot(X[k:k+2],Z[0:2],color='blue',marker='.',markersize = 10)
    axarr.grid(linestyle='--', linewidth=0.5, color='gray')
    
#    plt.ylim(g.node[(N,-N+1)]['value'],g.node[(N,N+1)]['value']) 
#    plt.xlim(-0.5,N+0.5)
#    ff.canvas.draw()
# plt.show()

    
def path(axarr,Y,K,n):
    global X
    Z = np.empty(N+1)

    for i in range(0,N+1):
        Z[i]=K
        
    axarr.plot(X[0:n+1],Y[0:n+1],marker='.',color='red',linewidth=3,markersize = 16)
    axarr.plot(X[0:N+1],Z[0:N+1],color='black',linewidth=2)
    axarr.grid(linestyle='--', linewidth=0.5, color='gray')
#    ff.canvas.draw()

def graph_stock():
    S=nx.Graph()
    for k in range(0,N):
        for l in range(-k+1,k+3,2):
            S.add_edge((k,l),(k+1,l+1))
            S.add_edge((k,l),(k+1,l-1))
            
    for n in S.nodes():
        k=n[0]
        l=n[1]-1
        S.node[n]['value']=S0*((1.0+b)**((k+l)/2))*((1.0+a)**((k-l)/2))
    return S

In [ ]:
def European_call_price(K):

    price = nx.Graph()    
    hedge = nx.Graph()
    S = graph_stock()

    for k in range(0,N):
            for l in range(-k+1,k+3,2):
                price.add_edge((k,l),(k+1,l+1))
                price.add_edge((k,l),(k+1,l-1))
                hedge.add_edge((k,l),(k+1,l+1))
                hedge.add_edge((k,l),(k+1,l-1))
    
    for l in range(-N+1,N+3,2):
        price.node[(N,l)]['value'] = np.maximum(S.node[(N,l)]['value']-K,0)
     
    for k in reversed(range(0,N)):
        for l in range(-k+1,k+3,2):
            price.node[(k,l)]['value'] = (price.node[(k+1,l+1)]['value']*p+price.node[(k+1,l-1)]['value']*q)/(1+r)        
    return price

In [ ]:
def European_call_hedge(K):

    price = European_call_price(K)
    hedge = nx.Graph()
    S = graph_stock()

    for k in range(0,N):
            for l in range(-k+1,k+3,2):
                hedge.add_edge((k,l),(k+1,l+1))
                hedge.add_edge((k,l),(k+1,l-1))
    
    for l in range(-N+1,N+3,2):
        hedge.node[(N,l)]['value'] = 1

    for k in reversed(range(0,N)):
        for l in range(-k+1,k+3,2):
            hedge.node[(k,l)]['value'] = (price.node[(k+1,l+1)]['value']-price.node[(k+1,l-1)]['value'])/(b-a)/(S.node[(k,l)]['value'])
    for l in range(-N+1,N+1,2):
        hedge.node[(N,l)]['value'] = hedge.node[(N-1,l+1)]['value'] 
        hedge.node[(N,l+2)]['value'] = hedge.node[(N-1,l+1)]['value'] 

    return hedge

In [ ]:
print('Underlying asset prices:') 
ff, axarr = plt.subplots(1,figsize=(12,10))

plot_tree(axarr,graph_stock())

Y = np.empty(N+1)

Y[0]=3.34;Y[1]=3.37;Y[2]=3.4;Y[3]=3.2;Y[4]=3.28;Y[5]=3.45

path(axarr,Y,K,5)

In [ ]:
print('European call prices:') 
ff, axarr = plt.subplots(1,figsize=(12,10))

plot_tree(axarr,European_call_price(float(K))) 

Y[0]=0.34;Y[1]=0.33;Y[2]=0.335;Y[3]=0.186;Y[4]=0.183;Y[5]=0.290

path(axarr,Y,0,5)

print('Price at time 0 of the European call option:',float("{0:.4f}".format(European_call_price(float(K)).node[(0,1)]['value']))) 

In [ ]:
ff, axarr = plt.subplots(1,figsize=(12,10))
print('Hedging strategy:') 
plot_tree(axarr,European_call_hedge(float(K)))